# 3-3 PSPNet のネットワーク構成と実装

## PSPNet を構成するモジュール
PSPNet は次の4つのモジュールから構成される．

<img src="../image/p143.png">

入力のサイズは 3 x 475 x 475（色チャネル x 高さ x 幅）であるが，実際にはミニバッチサイズだけまとめて入力するため，（batch_num x 色チャネル x 高さ x 幅）の4次元テンソルである．
ネットワークの各モジュールは次の通り．

### 1. Feature モジュール（Encoder モジュール）
このモジュールでは入力画像の特徴を抽出する．
ネットワークの詳細は3.4節で実装する．
このモジュールの出力は 2048 x 60 x 60 となる．
画像の特徴を捉えるためチャネル数が 2048 となる一方で，画像サイズ自体は 60 x 60 となっている．

### 2. Pyramid Pooling モジュール
「とあるピクセルの物体ラベルを求めるには，様々なスケールでそのピクセルの周囲の情報を必要とする」という考えに基づいて設計されている．
すなわち，物体ラベルを求めるために，そのピクセルの周辺情報だけでなく，さらに大きな範囲の画像情報が必要となる．  
そこで，このモジュールでは画像全体，画像の半分程度，画像の1/3程度，画像の1/6程度を占める4種類の広さの特徴量マップを用いる．
モジュールの出力は 4096 x 60 x 60 となる．

### 3. Decoder モジュール（アップサンプリングモジュール）
このモジュールには2つの目的がある．

1. 各21クラスに対する確信度を計算する  
4096チャネルの入力情報から，60 x 60 サイズの画像に対して各ピクセルの物体ラベルを推定する．  
出力データの値は各ピクセルが21クラスのそれぞれに属する確信度となっている．

1. アップサンプリング  
これまでのモジュールによって，画像サイズが小さくなっているため，画像サイズが 21 x 475 x 475 となるように元のサイズに変換する．

推論時には Decoder モジュールの出力を利用して，確率が最大となる物体クラスを各ピクセルのラベルとする．

### 4. AuxLoss モジュール
Aux は Auxiliary（補助の）の略．
このモジュールは損失関数を計算するときの「補助的な」役割を果たす．
Feature モジュールから途中のテンソルを抜き出し，そのテンソルを入力データとして Decoder モジュールと同じように各ピクセルに対応する物体ラベルを推定する．
入力は 1024 x 60 x 60，出力は 21 x 475 x 475 となる．学習時には Decoder と AuxLoss モジュールの出力の両方を正解情報と照合し損失を計算する．推論時にはこのモジュールは用いない．  
AuxLoss は Feature モジュールの中間出力でセグメンテーションを行うため，精度は低くなるが Feature モジュールの結合パラメータの学習に補助的な役割を果たす．

## PSPNet クラスの実装
PyTorch の nn.Module を継承し PSPNet のクラスを実装する．  
まず，コンストラクタで PSPNet の形を規定するパラメータを設定する．
その後各モジュールのオブジェクトを用意する．
Feature モジュールは feature_conv，feature_res_1，feature_res_2，feature_dilated_res_1，feature_dilated_res_2 の 5 つのサブネットワーク
から構成される．
その他のモジュールはそれぞれ 1 つのサブネットワークから構成される
クラス PSPNet のメソッドは forward のみで，順番に各モジュールのサブネットワークを実行する．
ただし，AuxLoss モジュールを Feature モジュールの4つ目のサブネットワーク feature_dilated_res_1 の後に挟み、その出力を変数 output_aux として作成し，メソッド forward の最後でメインの output と output_aux を返す。

In [2]:
import torch
from torch import nn

class PSPNet(nn.Module):
    def __init__(self, n_classes):
        super(PSPNet, self).__init__()
        
        # パラメータの設定
        block_config = [3, 4, 6, 3] # resnet50
        img_size = 475
        img_size_8 = 60             # img_size の 1/8
        
        # 4つのモジュールを構成するサブネットワークを用意
        # Feature モジュール
        self.feature_conv = FeatureMap_Convolution()
        self.feature_res_1 = ResidualBlockPSP(n_blocks=block_config[0], 
                                              in_channels=128, mid_channels=64, out_channels=256, stride=1, dilation=1)
        self.feature_res_2 = ResidualBlockPSP(n_blocks=block_config[1], 
                                              in_channels=256, mid_channels=128, out_channels=512, stride=2, dilation=1)
        self.feature_dilated_res_1 = ResidualBlockPSP(n_blocks=block_config[2], 
                                                      in_channels=512, mid_channels=256, out_channels=1024, stride=1, dilation=2)
        self.feature_dilated_res_2 = ResidualBlockPSP(n_blocks=block_config[3], 
                                                      in_channels=1024, mid_channels=512, out_channels=2048, stride=1, dilation=4)
        # Pyramid Pooling モジュール
        self.pyramid_pooling = PyramidPooling(in_channels=2048, pool_sizes=[6, 3, 2, 1], height=img_size_8, width=img_size_8)
        # Decoder モジュール
        self.decode_feature = DecodePSPFeature(height=img_size, width=img_size, n_classes=n_classes)
        # AuxLoss モジュール
        self.aux = AuxiliaryPSPlayers(in_channels=1024, height=img_size, width=img_size, n_classes=n_classes)
        
    def forward(self, x):
        x = self.feature_conv(x)
        x = self.feature_res_1(x)
        x = self.feature_res_2(x)
        x = self.feature_dilated_res_1(x)
        output_aux = self.aux(x)
        x = self.feature_dilated_res_2(x)
        x = self.pyramid_pooling(x)
        output = self.decode_feature(x)
        
        return (output, output_aux)